![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Text Classification with ClassifierDL

In [3]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)

spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.164.162.127:7077 
 AppName 
 Databricks Shell

## Load Dataset

In [5]:
! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv
! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv

In [6]:
dbutils.fs.cp("file:/databricks/driver/news_category_train.csv", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/news_category_test.csv", "dbfs:/")

Out[3]: True

In [7]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")

trainDataset.show(truncate=50)

+--------+--------------------------------------------------+
category| description|
+--------+--------------------------------------------------+
Business| Short sellers, Wall Street's dwindling band of...|
Business| Private investment firm Carlyle Group, which h...|
Business| Soaring crude prices plus worries about the ec...|
Business| Authorities have halted oil export flows from ...|
Business| Tearaway world oil prices, toppling records an...|
Business| Stocks ended slightly higher on Friday but sta...|
Business| Assets of the nation's retail money market mut...|
Business| Retail sales bounced back a bit in July, and n...|
Business|" After earning a PH.D. in Sociology, Danny Baz...|
Business| Short sellers, Wall Street's dwindling band o...|
Business| Soaring crude prices plus worries about the e...|
Business| OPEC can do nothing to douse scorching oil pr...|
Business| Non OPEC oil exporters should consider increa...|
Business| WASHINGTON/NEW YORK (Reuters) - The auction fo...|
Business| The dollar tumbled broadly on Friday after da...|
Business|If you think you may need to help your elderly ...|
Business|The purchasing power of kids is a big part of w...|
Business|There is little cause for celebration in the st...|
Business|The US trade deficit has exploded 19 to a recor...|
Business|Oil giant Shell could be bracing itself for a t...|
+--------+--------------------------------------------------+
only showing top 20 rows

In [8]:
trainDataset.count()


Out[5]: 120000

In [9]:
from pyspark.sql.functions import col

trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
category|count|
+--------+-----+
 World|30000|
Sci/Tech|30000|
 Sports|30000|
Business|30000|
+--------+-----+

In [10]:
testDataset = spark.read \
      .option("header", True) \
      .csv("news_category_test.csv")


testDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
category|count|
+--------+-----+
Business| 1900|
 Sports| 1900|
Sci/Tech| 1900|
 World| 1900|
+--------+-----+

In [11]:
# if we want to split the dataset
'''
(trainingData, testData) = trainDataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))
'''

## ClassiferDL with Word Embeddings and Text Preprocessing

In [13]:
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

glove_embeddings = WordEmbeddingsModel().pretrained() \
 .setInputCols(["document",'lemma'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("category")\
  .setMaxEpochs(3)\
  .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [15]:
# Train (8 min for 10 epochs)
clf_pipelineModel = clf_pipeline.fit(trainDataset)

In [16]:
clf_pipelineModel.stages[-1].write().overwrite().save('dbfs:/ClassifierDL_wordemb_e5')


In [18]:
# get the predictions on test Set

preds = clf_pipelineModel.transform(testDataset)


In [19]:
preds = clf_pipelineModel.transform(testDataset)

preds.select('category','description',"class.result").show(10, truncate=80)


+--------+--------------------------------------------------------------------------------+----------+
category| description| result|
+--------+--------------------------------------------------------------------------------+----------+
Business|Unions representing workers at Turner Newall say they are 'disappointed' af...| [World]|
Sci/Tech| TORONTO, Canada A second team of rocketeers competing for the #36;10 mil...| [World]|
Sci/Tech| A company founded by a chemistry researcher at the University of Louisville ...| [World]|
Sci/Tech| It's barely dawn when Mike Fitzpatrick starts his shift with a blur of color...|[Business]|
Sci/Tech| Southern California's smog fighting agency went after emissions of the bovin...| [World]|
Sci/Tech|"The British Department for Education and Skills (DfES) recently launched a "...| [World]|
Sci/Tech|"confessed author of the Netsky and Sasser viruses, is responsible for 70 per...| [World]|
Sci/Tech|\\FOAF/LOAF and bloom filters have a lot of interesting properties for socia...|[Business]|
Sci/Tech|"Wiltshire Police warns about ""phishing"" after its fraud squad chief was ta...| [World]|
Sci/Tech|In its first two years, the UK's dedicated card fraud unit, has recovered 36,...| [World]|
+--------+--------------------------------------------------------------------------------+----------+
only showing top 10 rows

In [20]:

preds_df = preds.select('category','description',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])


In [21]:
# due to bug in cluster mode (https://github.com/JohnSnowLabs/spark-nlp/issues/857) , as a workaround, you can just save the fitted model and then load back from dbfs and then transform on the test set. 

clf_pipelineModel.stages[-1].write().overwrite().save('dbfs:/ClassifierDL_wordemb_e5')

classsifierdlmodel_loaded = ClassifierDLModel.load('dbfs:/ClassifierDL_wordemb_e5')

clf_pipeline_pred = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            clf_pipelineModel.stages[-2],
            classsifierdlmodel_loaded])

empty_data = spark.createDataFrame([[""]]).toDF("description")

result = clf_pipeline_pred.fit(empty_data).transform(testDataset)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][OK!]

In [22]:
preds_df = result.select('category','description',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['category']))


precision recall f1-score support

 Business 0.83 0.83 0.83 1889
 Sci/Tech 0.86 0.84 0.85 1952
 Sports 0.96 0.94 0.95 1951
 World 0.86 0.91 0.89 1808

 micro avg 0.88 0.88 0.88 7600
 macro avg 0.88 0.88 0.88 7600
weighted avg 0.88 0.88 0.88 7600

## ClassifierDL with Universal Sentence Embeddings

In [24]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentece detector here if we want to train on and get predictions for each sentence

use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("category")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [25]:
use_pipelineModel = use_clf_pipeline.fit(trainDataset)
# 5 epochs takes around 10 min


## Getting prediction from Trained model

In [27]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(use_pipelineModel)

In [28]:
testDataset.select('description').take(2)

Out[26]: [Row(description="Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul."),
 Row(description=' TORONTO, Canada A second team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for privately funded suborbital space flight, has officially announced the first launch date for its manned rocket.')]

In [29]:
text='''
Fearing the fate of Italy, the centre-right government has threatened to be merciless with those who flout tough restrictions. 
As of Wednesday it will also include all shops being closed across Greece, with the exception of supermarkets. Banks, pharmacies, pet-stores, mobile phone stores, opticians, bakers, mini-markets, couriers and food delivery outlets are among the few that will also be allowed to remain open.
'''
result = light_model.annotate(text)

result['class']

Out[27]: ['Business']

### Saving the trained model

In [31]:
use_pipelineModel.stages

Out[28]: [DocumentAssembler_d2b783089b74,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 ClassifierDLModel_edcbc2b517cb]

In [32]:
use_pipelineModel.stages[2].write().overwrite().save('dbfs:/ClassifierDL_USE_20200923_e5')

In [33]:
use_classsifierdlmodel_loaded = ClassifierDLModel.load('dbfs:/ClassifierDL_USE_20200923_e5')


In [34]:
use_pipeline_pred = Pipeline(
    stages=[document,
            use_pipelineModel.stages[1],
            use_classsifierdlmodel_loaded])

empty_data = spark.createDataFrame([[""]]).toDF("description")

use_result = use_pipeline_pred.fit(empty_data).transform(testDataset)


In [35]:
preds_df = use_result.select('category','description',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['category']))


precision recall f1-score support

 Business 0.85 0.84 0.84 1925
 Sci/Tech 0.88 0.85 0.86 1964
 Sports 0.98 0.95 0.97 1959
 World 0.86 0.93 0.89 1752

 micro avg 0.89 0.89 0.89 7600
 macro avg 0.89 0.89 0.89 7600
weighted avg 0.89 0.89 0.89 7600